In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
stations_geo = pd.read_csv('Data/raw/stations_loc.csv', usecols=['NAME', 'the_geom'])
zip_by_station = pd.read_csv('Data/raw/zips.csv', dtype={'zip_code':'object'}, names=['STATION','ZIP'])
zip_by_borough = pd.read_csv('Data/raw/zip_by_borough.csv', dtype={'ZIP':'object'})
turnstile_stations = pd.read_csv('Data/Processed/unique_mta_stations.csv',index_col=0)

In [3]:
#We have geo data for 473 stations, but some have more than one set of coords, 
#so we'll only consider 355
stations_geo = stations_geo.drop_duplicates(subset=('NAME'), keep='last').reset_index(drop=True)

In [4]:
# zip_by_station contains name matches for the stations in tursntile data, but I want to add 
# the ones the author dropped in case I can match them to geo data I acquired.
stations_zip = turnstile_stations.merge(zip_by_station, on='STATION', how='left')

# Add borough column for easy filtering later
station_borough = stations_zip.merge(zip_by_borough, on='ZIP', how='left').drop_duplicates(subset=['STATION','ZIP','BOROUGH'])

#These stations have either been matched to worng zips outside or NYC or have been dropped
##station_borough[station_borough.BOROUGH.isna()] 
#I'll try to match some of these if I have time later
#station_borough.dropna(subset=['BOROUGH'], inplace=True) #None of these zips are in NYC, so I'll drop them
#brook_manh = station_borough[(station_borough.BOROUGH=='Brooklyn')|(station_borough.BOROUGH=='Manhattan')]

In [5]:
geo_test = stations_geo.copy(deep=True)
station_test = station_borough.copy(deep=True)

In [6]:
#stations_zip.info() #We have turnstile data for only 379 stations
#We have geo data for 473 stations, but some have more than one set of coords, 
#so we'll only consider 355
#stations_geo.info()    
#stations_geo = stations_geo.drop_duplicates(subset=['NAME'], keep='last').reset_index(drop=True)
stations_geo.head()

,NAME,the_geom
0,Astor Pl,POINT (-73.99106999861966 40.73005400028978)
1,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568)
2,238th St,POINT (-73.90087000018522 40.88466700064975)
3,Kingston - Throop Aves,POINT (-73.94085899871263 40.67991899941601)
4,65th St,POINT (-73.8987883783301 40.74971952935675)


In [7]:
# Edit station names in geo data to try to match turnstile data
# I want to remove the ordinal indicators from the names in the geo data, I will apply it before capitalizing
# to avoid deleting the street indicators ('St') and other data that are not ord indicators

stations_geo['NAME'] = (stations_geo.NAME.str.replace(' -','-')
                                        .str.replace('- ','-')
                                        .str.strip())
station_borough['STATION'] = (station_borough.STATION.str.strip().str.replace('AVE','AV')
                              .str.replace('STREET','ST').str.replace('COLLEGE','CO')
                              .str.replace('SQUARE','SQ').replace('STS','ST').replace('/','-'))

# I will use fuzzywuzzy, but it's not very accurate with the numbered streets so I will treat them separately.
stations_geo['NUMBER'] = stations_geo.NAME.apply(lambda x: True if True in [char.isdigit() for char in x] else False)


#geo stations WITH numbers
stations_geo_nr = stations_geo[stations_geo.NUMBER==True]
stations_geo_nr['NAME'] = (stations_geo_nr.NAME.apply(lambda x: x.replace('th','').replace('1st','1')
                                                                         .replace('2nd','2').replace('3rd','3')
                                                                         .upper().replace('STS','ST')
                                                                         .replace('AVE','AV').replace('STREET','ST')
                                                                         .replace('COLLEGE','CO').replace('SQUARE','SQ')
                                                                         .replace('LOWER EAST SIDE','').strip()))
stations_geo_no_nr = stations_geo[stations_geo.NUMBER==False]
stations_geo_no_nr['NAME'] = stations_geo_no_nr.NAME.apply(lambda x: x.upper().replace('STS','ST')
                                                                         .replace('AVE','AV').replace('STREET','ST')
                                                                         .replace('COLLEGE','CO').replace('SQUARE','SQ')
                                                                          .strip())



# do merge
new_stations_geo = pd.concat([stations_geo_no_nr, stations_geo_nr])
merged = station_borough.merge(new_stations_geo, left_on='STATION', right_on='NAME', how='outer')


# retrieve unmerged
unmerged_geo = (merged[(merged.NAME.notna())&(merged.STATION.isna())]
                .reset_index(drop=True).drop(['ZIP','STATION','BOROUGH','NUMBER'], axis=1,errors='ignore'))
unmerged_station = (merged[(merged.STATION.notna())&(merged.NAME.isna())]
                    .reset_index(drop=True).drop(['NAME','the_geom','NUMNER'], axis=1, errors='ignore'))
unmerged_station = unmerged_station[~unmerged_station.STATION.isin(['2 AV','5 AV'])]

In [8]:
match1 = []
score1 = []

match2 = []
score2 = []

match3 = []
score3 = []


for i in unmerged_geo.NAME:
        ratio = process.extract( i, unmerged_station.STATION, limit=1)
        match1.append(ratio[0][0])
        score1.append(ratio[0][1])
        
for i in unmerged_geo.NAME:
        ratio = process.extract( i, unmerged_station.STATION, limit=1, scorer=fuzz.partial_ratio)
        match2.append(ratio[0][0])
        score2.append(ratio[0][1])
        
for i in unmerged_geo.NAME:
        ratio = process.extract( i, unmerged_station.STATION, limit=1, scorer=fuzz.token_set_ratio)
        match3.append(ratio[0][0])
        score3.append(ratio[0][1])        

unmerged_geo['MATCH1'] = pd.Series(match1)
unmerged_geo['SCORE1'] = pd.Series(score1)

unmerged_geo['MATCH2'] = pd.Series(match2)
unmerged_geo['SCORE2'] = pd.Series(score2)

unmerged_geo['MATCH3'] = pd.Series(match3)
unmerged_geo['SCORE3'] = pd.Series(score3)

In [9]:
#unmerged_geo.sort_values('SCORE2').head(25)      ##TOKEN_SET_RATIO IS THE WAY TO GO.
unmerged_geo.drop(['MATCH1','SCORE1','MATCH2','SCORE2'], axis=1,  inplace=True)
new_merged_geo = (unmerged_geo[unmerged_geo.SCORE3>70]
                  .merge(unmerged_station, left_on='MATCH3', right_on='STATION', how='inner'))

new_merged_geo.drop(['MATCH3','SCORE3'], axis=1, inplace=True)

total_merged = pd.concat([new_merged_geo, merged[(merged.NAME.notna())&(merged.STATION.notna())]])
total_merged = total_merged.drop_duplicates(subset='STATION').drop(['NUMBER'], axis=1)                        
#add the stations we don't have geo data for, since their zip code is still usefull
final_stations = pd.concat([total_merged, station_borough[~station_borough.STATION.isin(total_merged.STATION)]])
final_stations.reset_index(drop=True, inplace=True)                     

In [33]:
final_stations.loc[100,'STATION'] = '2 AV'
final_stations.loc[100,'ZIP'] = '10003'
final_stations.loc[100,'BOROUGH'] = 'Manhattan'
final_stations.loc[131,'the_geom'] = 'POINT (-73.98177094440949 40.690648119969794)'
final_stations.loc[157,'the_geom'] = 'POINT (-73.95024799996972 40.71407200064717)'
final_stations.loc[95,'the_geom'] = 'POINT (-73.97334700047045 40.764510999755284)'

In [34]:
(final_stations.loc[:,['STATION','ZIP','BOROUGH','the_geom']]
 .to_csv('Data/Processed/zip_boro_geo.csv', index=False))

#### Code for comparison tests

In [12]:
final_stations[final_stations.ZIP=='10019']

,NAME,the_geom,STATION,ZIP,BOROUGH
95,5 AV-59 ST,POINT (-73.97334700047045 40.764810999755284),5 AV/59 ST,10019,Manhattan
119,7 AV,POINT (-73.98169782344476 40.76297015245628),7 AV,10019,Manhattan
147,50 ST,POINT (-73.98598400026407 40.76245599925997),50 ST,10019,Manhattan
260,57 ST,POINT (-73.98072973372128 40.76456552501829),57 ST,10019,Manhattan
353,NaN,NaN,NEWARK HW BMEBE,10019,Manhattan


In [31]:
final_stations[final_stations.STATION=='5 AV']

,NAME,the_geom,STATION,ZIP,BOROUGH
372,NaN,NaN,5 AV,10309,Staten


In [32]:
final_stations[final_stations.STATION.str.contains('5 AV')]

,NAME,the_geom,STATION,ZIP,BOROUGH
65,5 AV-53 ST,POINT (-73.9752485052734 40.76008683231326),5 AV/53 ST,10022,Manhattan
95,5 AV-59 ST,POINT (-73.97334700047045 40.764810999755284),5 AV/59 ST,10019,Manhattan
151,25 AV,POINT (-73.98682900011477 40.59770366695856),25 AV,11214,Brooklyn
242,75 AV,POINT (-73.83692369387158 40.71804465348743),75 AV,10309,Staten
372,NaN,NaN,5 AV,10309,Staten


In [15]:
final_stations.dropna(subset=['NAME'])[final_stations.dropna(subset=['NAME']).NAME.str.contains('2 AV')]

,NAME,the_geom,STATION,ZIP,BOROUGH
98,-2 AV,POINT (-73.98993800003434 40.72340166574911),72 ST-2 AV,NaN,NaN


In [18]:
station_test[station_test.STATION.str.contains('5 AV')]

,STATION,ZIP,BOROUGH
1,5 AV/59 ST,10019,Manhattan
126,25 AV,11214,Brooklyn
360,5 AV/53 ST,10022,Manhattan
388,75 AV,10309,Staten
640,5 AVE,10309,Staten


In [16]:
geo_test[geo_test.NAME.str.contains('7th Ave')]

,NAME,the_geom
42,67th Ave,POINT (-73.85286048434907 40.726505475813006)
253,7th Ave,POINT (-73.98169782344476 40.76297015245628)


In [17]:
stations_geo_nr[stations_geo_nr.NAME.str.contains('7 AV')]

,NAME,the_geom,NUMBER
42,67 AV,POINT (-73.85286048434907 40.726505475813006),True
253,7 AV,POINT (-73.98169782344476 40.76297015245628),True


In [19]:
station_borough[station_borough.STATION.str.contains('/')]

,STATION,ZIP,BOROUGH
1,5 AV/59 ST,10019,Manhattan
30,DELANCEY/ESSEX,10002,Manhattan
340,161/YANKEE STAD,10451,Bronx
360,5 AV/53 ST,10022,Manhattan
362,LEXINGTON AV/53,10022,Manhattan
453,LEXINGTON AV/63,10065,Manhattan
474,PAVONIA/NEWPORT,07310,NaN
475,CITY / BUS,10018,Manhattan
548,138/GRAND CONC,10451,Bronx
550,149/GRAND CONC,10451,Bronx


In [20]:
unmerged_station[unmerged_station.STATION.str.contains('2 AV')]

,STATION,ZIP,BOROUGH,NUMBER
78,72 ST-2 AV,NaN,NaN,NaN
79,86 ST-2 AV,NaN,NaN,NaN
80,96 ST-2 AV,NaN,NaN,NaN


In [21]:
stations_geo[stations_geo.NAME.str.contains('AVE')]

,NAME,the_geom,NUMBER
